In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import json
from threading import Thread,Lock,Event

In [2]:
headers = {
    'authority': 'www.bigbasket.com',
    'method': 'GET',
    'path': '/listing-svc/v2/products?type=pc&slug=foodgrains-oil-masala&page=1',
    'scheme': 'https',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Cookie': 'x-entry-context-id=100; x-entry-context=bb-b2c; _bb_locSrc=default; x-channel=web; _bb_loid=j:null; _bb_bhid=; _bb_nhid=1723; _bb_vid=MTI4MjA0NDYxMzQ=; _bb_dsevid=; _bb_dsid=; _bb_cid=1; _bb_aid=MzA4NTgxODk5Nw==; csrftoken=HhsvZkeZFNA7XdmHV5IIUT49QLIvvrXbMF4gCJoPWxShyovvBqwpje2fFCm84nEj; _bb_home_cache=4cca0b57.1.visitor; bb2_enabled=true; ufi=1; bigbasket.com=2f61e0d4-9d4f-4530-ba94-334049c46704; _gcl_au=1.1.2033074425.1718993968; _ga=GA1.1.1843003858.1718993968; jarvis-id=5d0d66fb-63a4-4685-a900-2dfc7b925de7; _ga_FRRYG5VKHX=GS1.1.1719002693.3.1.1719002693.60.0.0; csurftoken=egUF8w.MTI4MjA0NDYxMzQ=.1720255452916.XQjr+mZO4SgN4wjdV3jFFPEr7mvcgjkbbVDcgrCBe/w=',
    'Sec-Ch-Ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Microsoft Edge";v="126"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0',
    'Content-Type': 'application/json',
    'Newrelic': 'eyJ2IjpbMCwxXSwiZCI6eyJ0eSI6IkJyb3dzZXIiLCJhYyI6IjgzNzMwNCIsImFwIjoiMTgzNDk4NzAwMiIsImlkIjoiYmM1NzlmNzRiNjA1ZjkzYyIsInRyIjoiZWIzNTliYTAzZjg3NGFmNTA4MGI3YWY5NjRkN2QzNjIiLCJ0aSI6MTcyMDIyOTg3MTQyNH19',
    'Priority': 'u=1, i'
}

In [23]:
product_pages = []
lock = Lock()
flag=True
def scrap_page(page,lock):
    global flag
    url = f"https://www.bigbasket.com/listing-svc/v2/products?type=pc&slug=foodgrains-oil-masala&page={page}"
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        with lock:
            try:
                product_pages.append(data["tabs"][0]["product_info"].get("products",[]))
            except Exception as e:
                print(f"Error processing page {page}, item {j}: {e}")
    except Exception as e:
        print(f"Request failed for page {page}: {e}")
        flag=False

if __name__ == "__main__":
    all_threads = []
    number_threads = 5
    page = 192
    while True:
        try:
            for _ in range(number_threads):
                t = Thread(target=scrap_page, args=(page,lock))
                if not flag :
                    raise Exception("Page Limit Reached")
                all_threads.append(t)
                t.start()
                # print(page)
                page += 1
                
            for t in all_threads:
                t.join()
            
            all_threads = []
        except Exception as e:
            print(f"Total pages scraped: {len(product_pages)}")
            # print(product_info)
            break

Request failed for page 276: Expecting value: line 1 column 1 (char 0)
Request failed for page 277: Expecting value: line 1 column 1 (char 0)
Total pages scraped: 84


In [24]:
product_detail=[]
for page in product_pages:
    for product in page:
        id=product.get("id") if product.get("id") else "Unavailable"
        price=product["pricing"]["discount"].get("subscription_price") or "unavailable"
        Image_link=product.get("images") if product.get("images") else "Unavailable"
        Brand=product["brand"].get("name") if product["brand"].get("name") else "Unavailable"
        title=product.get("desc") if product.get("desc") else "Unavailable"
        w=product.get("w") if product.get("w") else "Unavailable"
        availability=product["availability"].get("avail_status") if product["availability"].get("avail_status") else "Unavailable"
        
        tlc,mlc,llc=product["category"].get("tlc_name"), product["category"].get("mlc_name"), product["category"].get("llc_name")
        category=f"{tlc}/ {mlc}/ {llc}"
        mag=product.get("magnitude") if product.get("magnitude") else "Unavailable"
        unit=product.get("unit") if product.get("unit") else "Unavailable"
        Ean_code= product.get("ean_code") if product.get("ean_code") else "Unavailable"
        
        product_detail.append({"Id":id,"EAN code":Ean_code,"Title":title,"Brand":Brand,"Magnitude":w,"Quantity":mag,"Unit":unit,"Price":price,"Availability":availability,"Category":category,"Image":Image_link})
        
print("done")
# print(product_detail)

done


In [25]:
keys=product_detail[0].keys()

with open("scraped_data.csv","a", newline='', encoding='utf-8') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    # dict_writer.writeheader()  # Uncomment if writing the header for the first time
    dict_writer.writerows(product_detail)

In [26]:
import pandas as pd
df=pd.read_csv("scraped_data.csv")
df

,Id,EAN code,Title,Brand,Magnitude,Quantity,Unit,Price,Availability,Category,Image
0,40075197,890537007116,Basmati Rice/Basmati Akki - Rozana Super 90,Daawat,5 kg,5000,g,349.0,1,"Foodgrains, Oil & Masala/ Rice & Rice Products...",[{'s': 'https://www.bigbasket.com/media/upload...
1,147492,890188716598,Gold Refined Cooking oil | Blended Rice Bran &...,Saffola,2 L,2000,ml,335.0,1,"Foodgrains, Oil & Masala/ Edible Oils & Ghee/ ...",[{'s': 'https://www.bigbasket.com/media/upload...
2,20005214,8904043926797,Fine Besan/Kadale Hittu - 100% Chana Dal,Tata Sampann,1 kg,1000,g,111.0,1,"Foodgrains, Oil & Masala/ Atta, Flours & Sooji...",[{'s': 'https://www.bigbasket.com/media/upload...
3,30010377,2000539,Urad Dal/Uddina Bele - Split,bb Popular,1 kg,1000,g,150.0,1,"Foodgrains, Oil & Masala/ Dals & Pulses/ Urad ...",[{'s': 'https://www.bigbasket.com/media/upload...
4,10000537,10000537,Walnut/Akhrot - Kernels,bb Popular,200 g,200,g,205.0,1,"Foodgrains, Oil & Masala/ Dry Fruits/ Other Dr...",[{'s': 'https://www.bigbasket.com/media/upload...
...,...,...,...,...,...,...,...,...,...,...,...
13169,40313633,40313633,Peanut Butter - Natural Crunchy,Yellow Bytes,350 g,350,g,299.0,1,"Foodgrains, Oil & Masala/ Dry Fruits/ Other Dr...",[{'s': 'https://www.bigbasket.com/media/upload...
13170,40314801,40314801,Refined Oil,GOLD PICK,1 L,1000,ml,150.0,10,"Foodgrains, Oil & Masala/ Edible Oils & Ghee/ ...",[{'s': 'https://www.bigbasket.com/media/upload...
13171,40241159,40241159,"Groundnut Oil - Cold Pressed, Kacchi Ghani, Pu...",Sneh-Sattva,5 L,5000,ml,1850.0,1,"Foodgrains, Oil & Masala/ Edible Oils & Ghee/ ...",[{'s': 'https://www.bigbasket.com/media/upload...
13172,40322532,762983176300,Kuttu Atta,EVAA Naturals,500 g,500,g,225.0,1,"Foodgrains, Oil & Masala/ Atta, Flours & Sooji...",[{'s': 'https://www.bigbasket.com/media/upload...


In [15]:
# for product in product_data[:100]:
print(product_pages[0])

[{'id': '40295240', 'desc': 'Sona Masoori Boiled - Rice', 'sku_max_quantity': 0, 'pack_desc': 'Bag', 'sort_index_pos': 4, 'cart_count': 0, 'is_best_value': False, 'magnitude': '26000', 'w': '26 kg', 'absolute_url': '/pd/40295240/bb-royal-sona-masoori-boiled-rice-26-kg-bag/', 'unit': 'g', 'usp': '', 'availability': {'avail_status': '001', 'display_mrp': True, 'display_sp': True, 'not_for_sale': False, 'button': 'Add', 'show_express': False}, 'pricing': {'discount': {'mrp': '2080', 'mrp_per_pack': None, 'd_text': '12% OFF', 'd_avail': 'true', 'prim_price': {'sp': '1828', 'sp_per_pack': None, 'icon': {'base_url': None, 'image': None, 'format': None}, 'desc': '', 'base_price': '70.31', 'base_unit': 'kg'}, 'sec_price': {'sp': None, 'icon': None, 'desc': None, 'background': None}, 'subscription_price': '1828', 'camp_detail': {'c': [2220539], 'c_f': 252, 'd_t': 'percent', 'd_v': 12.12, 'dual_pricing': False, 'm_f': 0, 'narq': 9999999, 'r_c': [2220539], 'v_f': 0}, 'offer_entry_text': '', 'offe